In [2]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
#https://www.youtube.com/watch?v=-1_PgWWmauHk&ab_channel=PythonEngineer

In [24]:



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_size = 128
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 1e-3

input_size = 28
sequence_length = 28
num_layers = 2


In [25]:
train_dataset = torchvision.datasets.MNIST(root='./data', train = True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train = False, transform=transforms.ToTensor(), download=True)

In [26]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [33]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [45]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        return self.fc(out)


In [46]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 1.1657
Epoch [1/2], Step [200/600], Loss: 0.6545
Epoch [1/2], Step [300/600], Loss: 0.6880
Epoch [1/2], Step [400/600], Loss: 0.5132
Epoch [1/2], Step [500/600], Loss: 0.3599
Epoch [1/2], Step [600/600], Loss: 0.2493
Epoch [2/2], Step [100/600], Loss: 0.3619
Epoch [2/2], Step [200/600], Loss: 0.2656
Epoch [2/2], Step [300/600], Loss: 0.1973
Epoch [2/2], Step [400/600], Loss: 0.2154
Epoch [2/2], Step [500/600], Loss: 0.4495
Epoch [2/2], Step [600/600], Loss: 0.2851


In [49]:
model.eval()
num_correct_prediction = 0
total_number_prediction = 0
for i, (images, labels) in enumerate(test_loader):
    images = images.reshape(-1, sequence_length, input_size)
    with torch.no_grad():
        output = model(images)
        predicted_labels = torch.argmax(output, dim=1)
        num_correct_prediction += torch.sum(labels == predicted_labels)
        total_number_prediction += len(labels)

print("Model : RNN")

print(f'Accuracy: {(num_correct_prediction/total_number_prediction):.2f}')


Model : RNN
Accuracy: 0.93


In [50]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, (h0,c0))
        out = out[:, -1, :]
        return self.fc(out)


In [52]:
model2 = LSTM(input_size, hidden_size, num_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

In [53]:

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        outputs = model2(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 0.6264
Epoch [1/2], Step [200/600], Loss: 0.6707
Epoch [1/2], Step [300/600], Loss: 0.2365
Epoch [1/2], Step [400/600], Loss: 0.0858
Epoch [1/2], Step [500/600], Loss: 0.2309
Epoch [1/2], Step [600/600], Loss: 0.1238
Epoch [2/2], Step [100/600], Loss: 0.2309
Epoch [2/2], Step [200/600], Loss: 0.1159
Epoch [2/2], Step [300/600], Loss: 0.1435
Epoch [2/2], Step [400/600], Loss: 0.0710
Epoch [2/2], Step [500/600], Loss: 0.1001
Epoch [2/2], Step [600/600], Loss: 0.0608


In [54]:
model.eval()
num_correct_prediction = 0
total_number_prediction = 0
for i, (images, labels) in enumerate(test_loader):
    images = images.reshape(-1, sequence_length, input_size)
    with torch.no_grad():
        output = model2(images)
        predicted_labels = torch.argmax(output, dim=1)
        num_correct_prediction += torch.sum(labels == predicted_labels)
        total_number_prediction += len(labels)

print("Model : LSTM")

print(f'Accuracy: {(num_correct_prediction/total_number_prediction):.2f}')


Model : LSTM
Accuracy: 0.97
